# CS506 Project Initial Report

**Chengyu Deng, Xiaotong Niu, Qian Zhang**


**Part 1 Data processing**

We use BERDO data to do the analysis.

2017: berdo2017.csv

2016: 2016-reported-energy-and-water-metrics.xlsx

2015: 2015-reported-energy-and-water-metrics.xlsx


In [1]:
import csv
import pandas as pd

# 2015 xlsx -> csv
data_2015_xlsx = pd.read_excel('2015-reported-energy-and-water-metrics.xlsx', index_col=None)
data_2015_xlsx.to_csv('berdo2015.csv')

# 2016 xlsx -> csv
data_2016_xlsx = pd.read_excel('2016-reported-energy-and-water-metrics.xlsx', index_col=None)
data_2016_xlsx.to_csv('berdo2016.csv')

# 2017
df_2015 = pd.read_csv('berdo2015.csv')
df_2016 = pd.read_csv('berdo2016.csv')
df_2017 = pd.read_csv('berdo2017.csv', encoding = "ISO-8859-1")


In [2]:
print(type(df_2015))
print(type(df_2016))
print(type(df_2017))
print(df_2015.shape)
print(df_2016.shape)
print(df_2017.shape)

print('-------------------------------')
print(list(df_2015.columns.values))
print('-------------------------------')
print(list(df_2016.columns.values))
print('-------------------------------')
print(list(df_2017.columns.values))
print('-------------------------------')

df_2016.drop('Years Reported', axis = 1, inplace = True)
df_2017.drop('Years Reported', axis = 1, inplace = True)

print()
print()
print(type(df_2015))
print(type(df_2016))
print(type(df_2017))
print(df_2015.shape)
print(df_2016.shape)
print(df_2017.shape)
print('-------------------------------')
print(list(df_2015.columns.values))
print('-------------------------------')
print(list(df_2016.columns.values))
print('-------------------------------')
print(list(df_2017.columns.values))
print('-------------------------------')

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
(1380, 23)
(1502, 24)
(1800, 24)
-------------------------------
['Unnamed: 0', 'Property Name', 'Reported', 'Property Type', 'Address', 'ZIP', 'Gross Area (sq ft)', 'Site EUI (kBTU/sf)', 'Energy Star Score', 'Energy Star Certified', 'Property Uses', 'Year Built', 'GHG Emissions (MTCO2e)', 'GHG Intensity (kgCO2/sf)', 'Total Site Energy (kBTU)', '% Electricity', '% Gas', '% Steam', 'Water Intensity (gal/sf)', 'Onsite Solar (kWh)', 'User Submitted Info', 'User Submitted Link', 'Tax Parcel']
-------------------------------
['Unnamed: 0', 'Property Name', 'Reported', 'Property Type', 'Address', 'ZIP', 'Gross Area (sq ft)', 'Site EUI (kBTU/sf)', 'Energy Star Score', 'Energy Star Certified', 'Property Uses', 'Year Built', 'GHG Emissions (MTCO2e)', 'GHG Intensity (kgCO2/sf)', 'Total Site Energy (kBTU)', '% Electricity', '% Gas', '% Steam', 'Water Intensity (gal/sf)', 'Onsite Solar

In [3]:
# Select properties which belong to BU
with open("BU_Property_List.csv", 'r') as propertyFile:
    reader = csv.reader(propertyFile, delimiter='\t')
    propertyList_convol = list(reader)
    
    propertyList = []
    for each in propertyList_convol:
        for element in each:
            propertyList.append(element)

print(len(propertyList))
print('------------------------------------')
print(propertyList)

# print(df.loc[df['B'].isin(['one','three'])])

df2015_BU = df_2015.loc[df_2015['Property Name'].isin(propertyList)]
df2016_BU = df_2016.loc[df_2016['Property Name'].isin(propertyList)]
df2017_BU = df_2017.loc[df_2017['Property Name'].isin(propertyList)]

print('------------------------------------')
print(df2015_BU.shape)
print(df2016_BU.shape)
print(df2017_BU.shape)

print('------------------------------------')
# print(df2015_BU)
# print(df2016_BU)
# print(df2017_BU)


62
------------------------------------
['785 Commonwealth Avenue (1 University Road)', '233 Bay State Road', "6-8 St. Mary's Street", '722-728 Commonwealth Avenue', '40-48 Buswell Street', '514-522 Park Drive', '110-112 Cummington Street', '700 Commonwealth Avenue', '64-86 Cummington Street', '22-24 Buswell Street', '10-18 Buswell Street', '213-217 Bay State Road', '631-639 Commonwealth Avenue', '605-615 Commonwealth Avenue (2 Silber Way)', '111 Cummington Street (664-666 Comm. Ave)', '30-38 Cummington Street', '20-28 Cummington Street (24 Cummington St)', '2 Cummington Street (23-25 Blandford Street)', '622-640 Commonwealth Avenue', '610 Commonwealth Avenue', '582-596 Commonwealth Avenue (590 Comm / 712 Bea)', '580 Commonwealth Avenue', 'Hotel Commonwealth', '631-639 Commonwealth Avenue', '605-615 Commonwealth Avenue (2 Silber Way)', '140 Bay State Road', '577-599 Commonwealth Avenue (595 Comm Ave)', '565-575 Commonwealth Avenue', '100 Bay State Road', '91 Bay State Road', '660 Beaco